In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
df = pd.read_csv('dataset/movies_metadata.csv', low_memory=False)
df.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [26]:
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [3]:
import preprocessing as pre
df = pre.clean_data(df)
df

c:\Users\Sveta\projects\preprocessing.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
c:\Users\Sveta\projects\preprocessing.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'] = pd.DatetimeIndex(df['release_date']).year


,title,genres,runtime,vote_average,vote_count,overview,id,year
0,Toy Story,"[animation, comedy, family]",81.0,7.7,5415.0,"Led by Woody, Andy's toys live happily in his ...",862,1995.0
1,Jumanji,"[adventure, fantasy, family]",104.0,6.9,2413.0,When siblings Judy and Peter discover an encha...,8844,1995.0
2,Grumpier Old Men,"[romance, comedy]",101.0,6.5,92.0,A family wedding reignites the ancient feud be...,15602,1995.0
3,Waiting to Exhale,"[comedy, drama, romance]",127.0,6.1,34.0,"Cheated on, mistreated and stepped on, the wom...",31357,1995.0
4,Father of the Bride Part II,[comedy],106.0,5.7,173.0,Just when George Banks has recovered from his ...,11862,1995.0
...,...,...,...,...,...,...,...,...
45461,Subdue,"[drama, family]",90.0,4.0,1.0,Rising and falling between a man and woman.,439050,NaN
45462,Century of Birthing,[drama],360.0,9.0,3.0,An artist struggles to finish his work while a...,111109,2011.0
45463,Betrayal,"[action, drama, thriller]",90.0,3.8,6.0,"When one of her hits goes wrong, a professiona...",67758,2003.0
45464,Satan Triumphant,[],87.0,0.0,0.0,"In a small town live two brothers, one a minis...",227506,1917.0


In [14]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df

,title,genres,runtime,vote_average,vote_count,overview,id,year,score
0,Toy Story,"[animation, comedy, family]",81.0,7.7,5415.0,"Led by Woody, Andy's toys live happily in his ...",862,1995.0,7.680728
1,Jumanji,"[adventure, fantasy, family]",104.0,6.9,2413.0,When siblings Judy and Peter discover an encha...,8844,1995.0,6.873806
2,Grumpier Old Men,"[romance, comedy]",101.0,6.5,92.0,A family wedding reignites the ancient feud be...,15602,1995.0,6.191313
3,Father of the Bride Part II,[comedy],106.0,5.7,173.0,Just when George Banks has recovered from his ...,11862,1995.0,5.685079
4,Heat,"[action, crime, drama, thriller]",170.0,7.7,1886.0,"Obsessive master thief, Neil McCauley leads a ...",949,1995.0,7.645616
...,...,...,...,...,...,...,...,...,...
9041,With Open Arms,[comedy],92.0,5.2,94.0,Jean-Étienne Fougerole is an intellectual bohe...,430365,2017.0,5.352812
9042,The Visitors: Bastille Day,[comedy],110.0,4.0,167.0,"Stuck in the corridors of time, Godefroy de Mo...",248705,2016.0,4.382375
9043,Titanic 2,"[action, adventure, thriller]",90.0,3.4,55.0,On the 100th anniversary of the original voyag...,44918,2010.0,4.475073
9044,In a Heartbeat,"[family, animation, romance, comedy]",4.0,8.3,146.0,A closeted boy runs the risk of being outed by...,455661,2017.0,7.609938


In [15]:
df['overview'].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Just when George Banks has recovered from his ...
4    Obsessive master thief, Neil McCauley leads a ...
Name: overview, dtype: object

In [16]:
tfidf = TfidfVectorizer(stop_words='english')
df['overview'] = df['overview'].fillna('')
tfidf_matrix = tfidf.fit_transform(df['overview'])

print(tfidf_matrix.shape)
print(tfidf.get_feature_names_out())

(9046, 28831)
['00' '000' '000th' ... 'óscar' 'ôtomo' 'über']


In [17]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
print(cosine_sim.shape)
print(cosine_sim[1])

(9046, 9046)
[0.01783579 1.         0.04839508 ... 0.         0.         0.        ]


In [18]:
#отображение индекса фильмов
mapping = pd.Series(df.index, index=df['title']).drop_duplicates()
mapping

title
Toy Story                         0
Jumanji                           1
Grumpier Old Men                  2
Father of the Bride Part II       3
Heat                              4
                               ... 
With Open Arms                 9041
The Visitors: Bastille Day     9042
Titanic 2                      9043
In a Heartbeat                 9044
Cadet Kelly                    9045
Length: 9046, dtype: int64

In [19]:
#Функция, которая принимает название фильма в качестве входных данных и выводит наиболее похожие фильмы.
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = mapping[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:15]
    movie_indices = [i[0] for i in sim_scores]

    return df['title'].iloc[movie_indices]

In [20]:
get_recommendations('The Godfather')

594      The Godfather: Part II
988     The Godfather: Part III
6979                 Blood Ties
4126                   Election
8765              Live by Night
1526             American Movie
5729                 Easy Money
2197                       Made
7198        The Look of Silence
6277                   Sinister
6237      The Cold Light of Day
3282          The Best of Youth
6907                        Joe
5830              Loose Cannons
Name: title, dtype: object